# Access the Processed (IMU+Mag+Index of each segment of 128 Hz) IMU Data File
# Access the Shot Segment Information
# Extract the Labeled Short Segments

In [1]:
# use annotation file to extract the active data points
import matplotlib as plt
import pandas as pd
import os
import numpy as np
import math
import csv

In [2]:
home_url = os.getcwd()
elan_url = os.path.abspath(os.path.join(home_url, '..', 'ELAN File'))
segment_info_url = os.path.abspath(os.path.join(home_url, '..', 'Segment Information'))
extracted_data_url = os.path.abspath(os.path.join(home_url, '..', 'Extracted Data Files'))

P1_file = "P1.csv"
P2_file = "P2-P3.csv"
P3_file = "P2-P3.csv"
P4_file = "P4.csv"

In [3]:
segment_info_url

'/home/avijoy/Downloads/Penalty Shoot-out Github/Segment Information'

In [4]:
P1_df = pd.read_csv(elan_url + "/" + P1_file)
P2_df = pd.read_csv(elan_url + "/" + P2_file)
P3_df = pd.read_csv(elan_url + "/" + P3_file)
P4_df = pd.read_csv(elan_url + "/" + P4_file)

## Shot Segment Information

In [5]:
P1_annotation = "P1.txt"
P2_annotation = "P2.txt"
P3_annotation = "P3.txt"
P4_annotation = "P4.txt"

In [6]:
list(P1_df)

['Unnamed: 0',
 'Timestamp',
 'Accelerometer X',
 'Accelerometer Y',
 'Accelerometer Z',
 'value',
 'New Index']

## Replace the Column Name

In [7]:
P1_df.rename(columns={'Unnamed: 0': 'Index', 'Accelerometer X': 'X', 'Accelerometer Y': 'Y', 'Accelerometer Z': 'Z'}, inplace=True)
P2_df.rename(columns={'Unnamed: 0': 'Index', 'Accelerometer X': 'X', 'Accelerometer Y': 'Y', 'Accelerometer Z': 'Z'}, inplace=True)
P3_df.rename(columns={'Unnamed: 0': 'Index', 'Accelerometer X': 'X', 'Accelerometer Y': 'Y', 'Accelerometer Z': 'Z'}, inplace=True)
P4_df.rename(columns={'Unnamed: 0': 'Index', 'Accelerometer X': 'X', 'Accelerometer Y': 'Y', 'Accelerometer Z': 'Z'}, inplace=True)

## Remove Redundant Columns

In [8]:
P1_df.drop(['Timestamp', 'value', 'New Index'], axis=1, inplace=True)
P2_df.drop(['Timestamp', 'value', 'New Index'], axis=1, inplace=True)
P3_df.drop(['Timestamp', 'value', 'New Index'], axis=1, inplace=True)
P4_df.drop(['Timestamp', 'value', 'New Index'], axis=1, inplace=True)

## Read Annotation and Create Separate Dataframe File
    - Tokenize each shot(data) entry
    - Aggregate different shots in different dataframes

In [9]:
# Read the text file
def read_annotation_file(df, file_name, user):
    lg_df = pd.DataFrame(columns=['X','Y','Z'])
    lt_df = pd.DataFrame(columns=['X','Y','Z'])
    rg_df = pd.DataFrame(columns=['X','Y','Z'])
    rt_df = pd.DataFrame(columns=['X','Y','Z'])
    mg_df = pd.DataFrame(columns=['X','Y','Z'])
    mt_df = pd.DataFrame(columns=['X','Y','Z'])

    annotation_file = open(file_name)
    for line in annotation_file:
#         print(line)
        tokens = line.split()
        start = int(tokens[4])+1
        end = int(tokens[7])+1
        
        item_df = df[start:end]
        if (tokens[0] == 'LG'):
            lg_df = pd.concat([lg_df,item_df], sort=True, axis = 0)
        elif (tokens[0] == 'LT'):
            lt_df = pd.concat([lt_df,item_df], sort=True, axis = 0)
        elif (tokens[0] == 'RG'):
            rg_df = pd.concat([rg_df,item_df], sort=True, axis = 0)
        elif (tokens[0] == 'RT'):
            rt_df = pd.concat([rt_df,item_df], sort=True, axis = 0)
        elif (tokens[0] == 'MG'):
            mg_df = pd.concat([mg_df,item_df], sort=True, axis = 0)
        elif (tokens[0] == 'MT'):
            mt_df = pd.concat([mt_df,item_df], sort=True, axis = 0)
    annotation_file.close()
    
    lg_df['Label'] = 0
    lt_df['Label'] = 1
    rg_df['Label'] = 2
    rt_df['Label'] = 3
    mg_df['Label'] = 4
    mt_df['Label'] = 5
    
    final_df = pd.concat([lg_df, lt_df, rg_df, rt_df, mg_df, mt_df], sort=True, axis = 0)
    final_df['User'] = user
    final_df.to_csv(extracted_data_url+ "/" + str(user)+'_raw_labeled.csv', float_format='%.6f')

In [10]:
read_annotation_file(P1_df, segment_info_url + "/" + P1_annotation, "P1")
read_annotation_file(P2_df, segment_info_url + "/" + P2_annotation, "P2")
read_annotation_file(P3_df, segment_info_url + "/" + P3_annotation, "P3")
read_annotation_file(P4_df, segment_info_url + "/" + P4_annotation, "P4")